# Aprendizem Profunda - DNN
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [11]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization, LeakyReLU, Input
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [12]:
df = pd.read_csv("data/dataset_final.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [13]:
def to_csv(results_df, name):
    os.makedirs("submissao2", exist_ok=True)
    path = f"submissao2/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [14]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [15]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 10000, Unique texts: 10000
Overlap between train and test after deduplication: 0


### Tokenizer

In [16]:
# Tokenize the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=100)
X_val = pad_sequences(X_val_seq, maxlen=100)
X_test = pad_sequences(X_test_seq, maxlen=100)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [17]:
data_set_professor = pd.read_csv("data/dataset3_inputs.csv", sep=';', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=100)  

### Early Stopping

In [18]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=10,            
    restore_best_weights=True,           
)

### Reduce_lr

In [19]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## DNN

In [20]:
model_dnn = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(64, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(32, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(1, activation="sigmoid"),
])

# Compile
model_dnn.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Train
history = model_dnn.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    shuffle=True
)

Epoch 1/200


/home/rui/miniconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6142 - loss: 1.0218 - val_accuracy: 0.9825 - val_loss: 0.5712 - learning_rate: 1.0000e-04
Epoch 2/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8567 - loss: 0.6231 - val_accuracy: 0.9744 - val_loss: 0.4598 - learning_rate: 1.0000e-04
Epoch 3/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9183 - loss: 0.5072 - val_accuracy: 0.9731 - val_loss: 0.4207 - learning_rate: 1.0000e-04
Epoch 4/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9425 - loss: 0.4443 - val_accuracy: 0.9725 - val_loss: 0.3933 - learning_rate: 1.0000e-04
Epoch 5/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9478 - loss: 0.4156 - val_accuracy: 0.9812 - val_loss: 0.3693 - learning_rate: 1.0000e-04
Epoch 6/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9643 - loss: 0.3850 - val_accuracy: 0.9819 - val_loss: 0.3526 - learning_rate: 1.0000e-04
Epoch 7/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.975

## Simple RNN

# Correr Modelos

**Test Set**

In [21]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_dnn, X_test, test_ids, "dnn")
model_dnn.save("models/dnn.keras")  

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


**Avaliação**

In [22]:
predict_and_save(model_dnn, X_professor, data_set_professor["ID"], "dnn_professor")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


## Classification Reports

In [23]:
print(classification_report(y_test, model_dnn.predict(X_test) > 0.5))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1000
         1.0       1.00      1.00      1.00      1000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

